<a href="https://colab.research.google.com/github/Maagnitude/CRC-slides-models/blob/main/CRC-slides-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα αναπτύξουμε **μοντέλα Συνελικτικών Νευρωνικών Δικτύων**, για να κατηγοριοποιήσουμε όσο πιο σωστά γίνεται τις 7180 εικόνες μικροσκοπίου, στις 9 κλάσεις που έχουμε.

**Αρχίζοντας...**

# **Βιβλιοθήκες**
Κάνουμε import τα απαραίτητα **modules**. 
*   Την **pandas** και την **numpy** για την διαχείρηση των δεδομένων μας.

*  Την **matplotlib.pyplot** και την **seaborn** για την οπτικοποίηση των δεδομένων μας. **Heatmaps** κλπ.

*  Το **tensorflow**, και από αυτό, τα **keras** και **layers** για την ανάπτυξη νευρωνικών δικτύων. 
  * Επίσης τα **datasets** του **keras** για να χωρίσουμε το **dataset** μας σε **train/validation/test**.

*  Το **EarlyStopping** για να εισάγουμε πρόωρο τερματισμό στην εκπαίδευση του μοντέλου, μέσω **callback**.

* Επίσης το **Rescaling** για να κάνουμε scale τις τιμές των εικόνων, και το **MaxPooling2D** για να εισάγουμε επίπεδα συγκέντρωσης στο δίκτυο.

* Το **EfficientNetB0** για εκπαιδεύσουμε το προεκπαιδευμένο αυτό δίκτυο στo dataset μας.

*  Τέλος, κάνουμε import τα **warnings** και τα φιλτράρουμε, ώστε να μην εμφανίζονται.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras.datasets
from keras.callbacks import EarlyStopping
from keras.layers import Rescaling
from keras.layers import MaxPooling2D

from keras.applications.efficientnet import EfficientNetB0

import warnings
warnings.filterwarnings(action='ignore')

## **drive mount και μεταφόρτωση των εικόνων**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/MyDrive/Colab Notebooks/CRC_slides.tar.gz' .
!tar -xvzf 'CRC_slides.tar.gz'
data_dir = '/content/CRC_slides'

# **Υλοποίηση συνάρτησης για την δημιουργία των datasets**
Περνάμε τα αρχεία του φακέλου **data_dir** στην μεταβλητή **all_ds** και ύστερα τα χωρίζουμε σε **training**, **validation** και **test sets**. Το πετυχαίνουμε με την χρήση των συναρτήσεων **next** και **skip**, όπου παίρνουμε τον αριθμό των αρχείων που αντιστοιχεί στο **devel_ds** (**next**), ύστερα τα κάνουμε **skip**, ώστε να πάρουμε τα επόμενα για το **train_ds** κτλ.

Σχετικά με τις παραμέτρους: 
* **data_dir** είναι ο φάκελος που βρίσκονται τα αρχεία.
* **labels='inferred'** για να παράξει τα labels από τους υποφακέλους.
* **label_moded='int'** που συμαίνει ότι τα labels είναι κωδικοποιημένα ως integers, και θα χρησιμοποιήσουμε "**sparse_categorical_crossentropy**".
  * Στην αρχή το είχα βάλει '**categorical**' και τα μοντέλα μου τρέχανε, με loss='**categorical_crossentropy**' αλλά το **EfficientNetB0** δεν έτρεχε, οπότε το γύρισα σε '**int**', και πλέον τρέχει και αυτό, και τα δικά μου με **sparse**.
* **class_names=None** για να χρησιμοποιηθεί αλφαριθμητικη σειρά, μιας και δεν μας νοιάζει.
* **color_mode='rgb'** για να έχουν 3 κανάλια οι εικόνες. (default value)
* **batch_size** βάζουμε το batch size που μας δίνει η εκφώνηση. Πόσες εικόνες θα πάρει μαζί. Το default θα ήταν 32.
* **image_size** βάζουμε το size που μας δίνει η εκφώνηση. Αν βάλουμε άλλο αριθμό απ αυτόν που είναι η εικόνες, γίνεται resize.
* **shuffle=True** που είναι και η default τιμή, απλά το βάζουμε να φαίνεται. Αν ήταν False θα έκανε sort τα δεδομένα σε αλφαριθμητική σειρά.
* **seed=123** ως σπόρο για το shuffling.

In [ ]:
def load_dataset(data_dir, train_pct=0.6, val_pct=0.2, test_pct=0.2, batch_size=64, img_size=(224, 224)):

  # Δημιουργία του training set
  train_ds = keras.utils.image_dataset_from_directory(data_dir, labels='inferred', label_mode='int', class_names=None, 
                              color_mode='rgb', validation_split=val_pct+test_pct, subset='training',  batch_size=batch_size, 
                              image_size=img_size, shuffle=True, seed=123)
  # Δημιουργία του αρχικού validation-test set
  val_ds = keras.utils.image_dataset_from_directory(data_dir, labels='inferred', label_mode='int', class_names=None, 
                              color_mode='rgb', validation_split=val_pct+test_pct, subset='validation',  batch_size=batch_size, 
                              image_size=img_size, shuffle=True, seed=123)

  # Χωρίζουμε το val_ds στα batches του, και ύστερα βάζουμε τα πρώτα μισά στο testing set, και τα υπόλοιπα στο validation set
  val_batches = tf.data.experimental.cardinality(val_ds)
  test_ds = val_ds.take(val_batches // 2)
  val_ds = val_ds.skip(val_batches // 2)

  # Το development set είναι ολόκληρο το dataset, χωρίς το test set
  devel_ds = tf.data.Dataset.concatenate(train_ds, val_ds)

  # Παίρνουμε τις κατηγορίες των δεδομένων
  classes = train_ds.class_names

  return devel_ds, train_ds, val_ds, test_ds, classes

In [ ]:
devel_ds, train_ds, val_ds, test_ds, classes = load_dataset(data_dir)

## **Για να δούμε τις πρώτες 9 εικόνες του train_ds**

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in devel_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(classes[labels[i]])
    plt.axis("off")

Εδώ τυπώνουμε το πόσες εικόνες έχει το κάθε batch, το μέγεθος κάθε εικόνας με τον αριθμό καναλιών, καθώς επίσης και τον αριθμό των labels που έχει το κάθε batch.

In [ ]:
for image, label in train_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy().shape)

In [ ]:
num_classes= len(classes)

## **Παρακάτω οπτικοποιούμε τον αριθμό των αρχείων κάθε κατηγορίας στο σύνολο ανάπτυξης**.

Παρατηρούμε ότι η κατανομή των κατηγοριών στο σύνολο δεδομένων μας είναι αρκετά **ανισόρροπη**, με ορισμένες κατηγορίες (**1η, 4η, 5η και 9η**) να έχουν σημαντικά περισσότερα δείγματα από άλλες (**3η, 6η, 7η και 8η**). Αυτό μπορεί να είναι προβληματικό κατά την εκπαίδευση μοντέλων μηχανικής μάθησης, καθώς το μοντέλο μπορεί να καταλήξει να υπερεκπαιδευτεί (**overfitting**) στις πιο διαδεδομένες κατηγορίες και να μην γενικεύει καλά στις κατηγορίες που υποεκπροσωπούνται.

In [ ]:
y = np.concatenate([y for x, y in devel_ds])
plt.hist(y, list(range(num_classes + 1)))
plt.show()

In [ ]:
y = np.concatenate([y for x, y in train_ds])
plt.hist(y, list(range(num_classes + 1)))
plt.show()

In [ ]:
y = np.concatenate([y for x, y in test_ds])
plt.hist(y, list(range(num_classes + 1)))
plt.show()

In [ ]:
y = np.concatenate([y for x, y in val_ds])
plt.hist(y, list(range(num_classes + 1)))
plt.show()

# **Υλοποίηση συνάρτησης για την δημιουργία συνελικτικού νευρωνικού δικτύου**.

Αρχικά περνάμε στην είσοδο την εικόνα **224x224 pixel x3** για το rgb.

Στο **πρώτο layer** κάνουμε rescaling τις τιμές των εικόνων (pixel values) από **[0, 255]** σε **[0, 1]**.

Στο επόμενο έχουμε ένα συνελικτικό επίπεδο **8 φίλτρων** 3x3 με **padding='same'** για να έχουμε έξοδο ίση με την είσοδο, και activation function **ReLU**.

Ύστερα έχουμε ένα επίπεδο **MaxPooling** με βήμα 2, βάζοντας **pool_size=(2, 2)** όπου αυτή θα είναι και η default τιμή των **strides** από τη στιγμή που δεν τους βάζουμε άλλη τιμή εμείς.

Επαναλαμβάνουμε τα δύο παραπάνω επίπεδα, με την διαφορά ότι το συνελικτικό τώρα έχει **16 φίλτρα**.

Ένα επίπεδο **Flatten()** για να μετατρέψουμε τις τιμές κάθε εικόνας σε διάνυσμα.

Τέλος, έχουμε ένα πλήρως συνδεδεμένο επίπεδο **32 νευρώνων** με activation function **ReLU**, κι ένα επίπεδο εξόδου για τις **9** (num_classes) **κατηγορίες** που θέλουμε να προβλέψουμε, με activation function **softmax** για να πάρουμε τιμές που αθροίζουν στο 1 (για το πόσο σίγουρο είναι το μοντέλο για κάθε κατηγορία).

In [ ]:
def cnn1(num_classes):

  model = keras.Sequential([
      keras.Input(shape=(224, 224, 3)),
      layers.Rescaling(1./255),
      layers.Conv2D(8, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Flatten(),
      layers.Dense(32, activation='relu'),
      layers.Dense(num_classes, activation='softmax')
  ])

  return model

## **Εκπαίδευση μοντέλου και εκτίμηση.**
Καλούμε την παραπάνω συνάρτηση και μας επιστρέφει το μοντέλο μας.

Το κάνουμε compile με Adam optimizer, βάζοντας τις επιθυμητές τιμές στις παραμέτρους. **Ρυθμό εκμάθησης=0.001**, **ρυθμό ενημέρωσης πρώτης ροπής=0.9** και **ρυθμό ενημέρωσης δεύτερης ροπής=0.99**. Για loss επέλεξα την **Sparse categorical crossentropy**, για να μην κάνω one-hot encoding στα labels.

Ρυθμίζουμε την εκπαίδευση του μοντέλου να σταματήσει πρόωρα αν δεν παρουσιαστεί μείωση της απώλειας στο σύνολο επικύρωσης για 5 εποχές, με την κλήση της **EarlyStopping** δίνοντας της ως παράμετρο την τιμή που πρέπει να παρακολουθεί και τις εποχές πέραν των οποιών θα πρέπει να σταματήσει η εκπαίδευση. Την περνάμε ως **callback** στην fit.

Τέλος, εκπαιδεύουμε το μοντέλο και το εξετάζουμε στο σύνολο επικύρωσης.

In [ ]:
model = cnn1(num_classes)

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(train_ds, validation_data=val_ds, batch_size=64, epochs=20, callbacks=[early_stopping])
score = model.evaluate(val_ds)

## **Αποτελέσματα**
Την πρώτη φορά το μοντέλο εκπαιδεύτηκε για **42 λεπτά**, για τις **18 εποχές** μιας και χρειάστηκε να τερματιστεί πρόωρα, κι αυτό γιατί απ την 13η εποχή που είχε **val_loss: 0.4994**, στις επόμενες 5 εποχές δεν κατάφερε να πέσει κάτω απ αυτή την τιμή (η οποία ήταν η χαμηλότερη, από κάθε άλλη). Ωστόστο η εξέταση στο σύνολο επικύρωσης έφερε τα τελικά αποτελέσματα **loss: 0.5732** και **accuracy: 0.7985**

Επειδή χρειάστηκε να ξανακάνω κάποια πράγματα, έπρεπε να το ξανατρέξω, αυτή τη φορά με την **TPU** του **Google Colab**. Έτρεξε και τις **20 εποχές**, σε **47 λεπτά**. Τα αποτελέσματα ήταν **loss: 0.4546** και **accuracy: 0.8586**, αρκετά παραπάνω από την προηγούμενη φορά. Ας δούμε τώρα πως θα τα πάει στο **test set**.

In [ ]:
def confusion_matrix(model, test_ds):
    y_test = []
    y_pred = []
    for x_1, y_1 in test_ds:
        y_pred_1 = model.predict(x_1)
        y_test.append(y_1)
        y_pred.append(y_pred_1)
        y_true = np.concatenate(y_test)
        y_p = np.concatenate(y_pred)
        y_hat = tf.argmax(y_p, axis=1)
        cm = tf.math.confusion_matrix(y_true, y_hat)
    return cm, y_hat, y_true

## **Υλοποίηση συνάρτησης για τον σχεδιασμό του Confusion Matrix**

Με χρήση ενός heatmap θα παρουσιάσουμε τα αποτελέσματα του υπολογισμένου confusion matrix, όπου βάζουμε τις παραμέτρους annot=True και annot_kws={'size':20} για να εμφανίσουμε τις τιμές σε κάθε περιοχή του πίνακα, και να έχουν μέγεθος 20. Επίσης, η παράμετρος **fmt="d"** είναι για να εμφανίζονται οι τιμές ως ακέραιοι. (Γιατί χωρίς αυτήν, το 466 εμφανιζόταν ως 4.7e+02)

Τέλος, με την **axis.tick_top()** βάζουμε τα labels του άξονα x στην κορυφή του plot.

In [ ]:
def heat_confmatrix (cm):
  
  labels = classes

  # Δίνουμε τα labels στον πίνακα
  cm_plt = pd.DataFrame(cm, index = labels, columns = labels)
  
  plt.subplots(figsize=(9, 7))
  
  ax = sns.heatmap(cm_plt, cmap='viridis', annot=True, annot_kws={'size':20}, fmt="d")

  ax.xaxis.tick_top()
  ax.set_title("Confusion Matrix\n")
  plt.ylabel('True')
  plt.xlabel('Predicted')
  plt.show()

Παίρνουμε και το **y_true** ώστε να υπολογίσουμε παρακάτω την μετρική **f1_score**.

In [ ]:
cm, y_hat, y_true = confusion_matrix(model, test_ds)

## **f1_score**
Θα χρησιμοποιήσουμε την μετρική **F1 score**, δίνοντας της ως παραμέτρους τις προβλέψεις **y_hat**, τις πραγματικές τιμές **y_true** και το average='micro' ώστε να υπολογιστεί συνολικά μετρώντας το σύνολο των **αληθώς θετικών**, των **ψευδώς αρνητικών** και των **ψευδώς θετικών** αποτελεσμάτων.

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_true, y_hat, average='micro')

# (Διαγώνιος / Όλες τις τιμές) -> 1181 / 1408 = ~0.83877

## **Ορθότητα μοντέλου**
Η ορθότητα του μοντέλου μας στο σύνολο **εκπαίδευσης** έφτασε μέχρι κα το **0.9942**. Στην τελευταία εποχή ήταν **0.9870**. Στο σύνολο **επικύρωσης** έφτασε το **0.8586** και στο σύνολο **δοκιμής** το **~0.8388**, όπως υπολογίσαμε με την **f1_score**.

## **Confusion Matrix**
Στις παρακάτω **4 κατηγορίες**, ενώ λόγω της **ανισορροπίας** περιμέναμε κακές προβλέψεις, το μοντέλο τα πάει αναπάντεχα καλά.

* Παρατηρούμε ότι για την κατηγορία **STR** δεν την έχει μάθει καλά, μιας και μάντεψε σωστά σε **34 δείγματα**, και μάντεψε λανθασμένα σε (7+1+4+14+8+16): **50 δείγματα**, τις κατηγορίες DEB, LYM, MUC, MUS, NORM, TUM αντίστοιχα, δίνοντας ποσοστό ακρίβειας (34 / 84): **0.4047**

* Την **DEB** την πέτυχε σε **54 δείγματα** και την μπέρδεψε με άλλες σε 14 δείγματα, δίνοντας ποσοστό (54 / 68): ~**0.7941**

* Στην **MUS** πέφτει σε **83 δείγματα** μέσα, και μπερδεύει μόνο 4 για DEB, 20 για STR, και 3 για TUM δίνοντας ποσοστό ακρίβειας (83 / 110): ~**0.7545**

* Τέλος, στην κατηγορία **NORM** τα πάει επίσης καλύτερα από άλλη φορά που το έτρεξα, καθώς προβλέπει **98 δείγματα** σωστά, και στα 26 δείγματα προβλέπει TUM, και στα υπόλοιπα (1+1+7+1+7): **17 δείγματα**, προβλέπει αντίστοιχα DEB, LYM, MUC, MUS και STR. Ποσοστό ακρίβειας (98 / 141): ~**0.695**

Την προηγούμενη φορά που το έτρεξα (και είχε τερματίσει στην **18η εποχή** λόγω **Early_stopping**) είχε ποσοστά ακρίβειας -> **STR: ~0.217**, **DEB: ~0.507**, **MUS: ~0.854** και **NORM: ~0.276**

Οι κατηγορίες που παρατηρήθηκε το μικρότερο ποσοστό ακρίβειας (κυρίως στην προηγούμενη εκπαίδευση), είναι οι ίδιες με αυτές που παρατηρήσαμε ότι υπάρχουν σε μικρή ποσότητα σε όλα τα sets (**3η, 6η, 7η και 8η**), που σημαίνει ότι πολύ πιθανόν το μοντέλο **υποεκπαιδεύτηκε** (**underfitting**) σε αυτές.

Τέλος, στις κατηγορίες **ADI**, **BACK**, **MUC** και **TUM** (**1η, 4η, 5η και 9η**), τα πήγε πολύ καλά, μιας και όπως είδαμε και πιο πάνω, υπάρχουν πολλά δείγματα αυτών των κατηγοριών στο dataset, με αποτέλεσμα το μοντέλο να τις μάθει πολύ καλύτερα απ' τις άλλες, και ακόμα να μπερδεύει πολλές φορές και τις άλλες με αυτές, όπως παρατηρήσαμε.

In [ ]:
heat_confmatrix(cm.numpy())

## **Εναλλακτική μετρική**
Αν δούμε τα δείγματα ως: 

[**ADI**, **BACK**, **DEB**, **LYM**, **MUC**, **MUS**, **NORM** -> **NORMAL**] και [**STR**, **TUM** -> **NOT NORMAL**]

Ο πίνακας σύγχυσης θα είναι πλέον **2x2** και σύμφωνα με υπολογισμούς που έκανα, θα έχει τις τιμές:

                       PREDICTIONS
                   NORMAL  / NOT NORMAL
                /----------|------------\
         NORMAL |   1007   |     69     |            
                |----------|------------| TRUE
     NOT NORMAL |    72    |    260     |
                \----------|------------/

Εμάς ουσιαστικά μας ενδιαφέρουν οι τιμές **69** και **72**. Σημαίνουν πως σε 69 δείγματα είχαμε υγιή ιστό και το μοντέλο πρόβλεψε ότι δεν είναι υγιής, ενώ στα 72 δεν είχαμε υγιή ιστό και το μοντέλο πρόβλεψε ότι είναι υγιής.

Άρα σε ένα ποσοστό (72 / 141): ~**0.5106** κάνει λάθος το οποίο οδηγεί στην 1η περίπτωση όπου ένας **μη υγιής** ιστός, θα **θεωρηθεί υγιής** και δεν θα προχωρήσει σε θεραπεία.

και

σε ένα ποσοστό ~**0.4893** κάνει λάθος το οποίο οδηγεί στην 2η περίπτωση όπου ένας ιστός **υγιής** θα **θεωρηθεί μη υγιής** και θα προχωρήσει σε θεραπεία.

Με λίγα λόγια, αν κάνει λάθος σχετικά με το αν ένας ιστός είναι υγιής ή όχι, στην 1η περίπτωση θεωρεί **λανθασμένα** κάποιον **υγιή** με ποσοστό **0.5106** το οποίο είναι **μεγαλύτερο** απ' το ποσοστό **0.4893**, της 2ης περίπτωσης, όπου θεωρεί **λανθασμένα** κάποιον **μη υγιή**, και αυτό είναι αρκετά **αρνητικό** με την έννοια ότι το να χρειαστεί εσφαλμένα να **θεραπεύσουμε έναν ήδη υγιή** είναι **λιγότερο σημαντικό** από το, εσφαλμένα, να **μην θεραπεύσουμε έναν μη υγιή**. Αυτό που θα θέλαμε λοιπόν, είναι, το ποσοστό της 1ης περίπτωσης να είναι όσο μικρότερο γίνεται από αυτό της 2ης. Αν σας έπιασε πονοκέφαλος, απολογούμαι.

Δεν γνωρίζω αν είναι σωστό το σκεπτικό μου με τον πίνακα, και αν είναι **σωστή** και **τεκμηριωμένη** η εξήγηση, αλλά έτσι το σκέφτηκα, με βοήθεια και από το παράδειγμα που δόθηκε στο μάθημα της **Μηχανικής Μάθησης** (στις 13/12/2022)

**Σημ**: Λογικά θα υπάρχει συνάρτηση που θα το υπολογίζει κατευθείαν. Δεν την βρήκα όμως και επειδή είχα αυτόν τον πίνακα στο μυαλό μου, είπα να το φτιάξω έτσι.


# **4. Deeper CNN**
Το παρακάτω μοντέλο υλοποιήθηκε όπως το προηγούμενο, με την διαφορά ότι έχει **περισσότερα επίπεδα**, των οποίων τα φίλτρα είναι **3x3**, έχουν **padding='same'** για να χουμε ίδιο μέγεθος εξόδου με την είσοδο, και activation function την **ReLU**, και είναι τα εξής:
* 2 επίπεδα **32** φίλτρων
* 1 επίπεδο συγκέντρωσης Max Pooling με βήμα 4
* 2 επίπεδα **64** φίλτρων
* 1 επίπεδο συγκέντρωσης Max Pooling με βήμα 2
* 2 επίπεδα **128** φίλτρων
* 1 επίπεδο συγκέντρωσης Max Pooling με βήμα 2
* 3 επίπεδα **256** φίλτρων
* 1 επίπεδο συγκέντρωσης Max Pooling με βήμα 2
* 1 επίπεδο **512** φίλτρων
* 1 επίπεδο συγκέντρωσης Max Pooling με βήμα 2
* 1 επίπεδο μετατροπής σε 1 διάσταση
* 1 πλήρως συνδεδεμένο επίπεδο **1024** νευρώνων
* Τέλος, ένα επίπεδο με εξόδους τις κατηγορίες μας και activation function **softmax**.

In [ ]:
def cnn2(num_classes):

  model = keras.Sequential([
      keras.Input(shape=(224, 224, 3)),
      layers.Rescaling(1./255),
      layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),      
      layers.MaxPooling2D(pool_size=(4, 4)),
      layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),      
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),  
      layers.MaxPooling2D(pool_size=(2, 2)),   
      layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
      layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),  
      layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),  
      layers.MaxPooling2D(pool_size=(2, 2)),  
      layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),               
      layers.MaxPooling2D(pool_size=(2, 2)),  
      layers.Flatten(),
      layers.Dense(1024, activation='relu'),
      layers.Dense(num_classes, activation='softmax')
  ])

  return model

Εκτελέστηκαν οι 17 εποχές σε **3 ώρες και 16 λεπτά**, και τερματίστηκε. Προφανώς δεν τερμάτισε λόγω του **val_loss** μιας και δεν παρατηρείτε να πέρασαν 5 εποχές από την χαμηλότερη τιμή (**0.3434**). Αφήνω το κελί αυτό με τα αποτελέσματα του, και θα το ξανατρέξω πιο κάτω μπας και ολοκληρωθούν οι 20 εποχές για να είμαστε πιο ακριβείς. 


In [ ]:
model2 = cnn2(num_classes)

model2.summary()

model2.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model2.fit(train_ds, validation_data=val_ds, batch_size=64, epochs=20, callbacks=[early_stopping])
score = model2.evaluate(val_ds)

In [ ]:
model3 = cnn2(num_classes)

model3.summary()

model3.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model3.fit(train_ds, validation_data=val_ds, batch_size=64, epochs=20, callbacks=[early_stopping])
score = model3.evaluate(val_ds)

## **Τι παρατηρούμε για την επίδοση**
Για αρχή να σημειωθεί ότι τη δεύτερη φορά που το έτρεξα, τερματίστηκε πρόωρα στην **15η εποχή**, μετά από **4μιση ώρες** με **TPU**, λόγω του ότι μετά την **10η εποχή** όπου είχε **val_loss: 0.5189**, δεν ξαναέπεσε κάτω από αυτή την τιμή για τις επόμενες 5 εποχές.

Η παρατήρηση που μπορούμε να κάνουμε εδώ είναι ότι μεταξύ αυτού του δικτύου και του προηγούμενου, **υπάρχει διαφορά στην επίδοση**. Ακόμα και να συνέχιζε αυτό το μοντέλο, μπορούμε να δούμε ότι γενικά το **validation accuracy** του **ανεβοκατέβαινε** λίγο. Ενώ του προηγούμενου μοντέλου, ανέβαινε σχετικά σταθερά. Ακόμα και την πρώτη φορά που είχα τρέξει το προηγούμενο, και είχε accuracy **0.7985** ανέβαινε πάλι **σταθέρα**, και πάλι τα πήγε καλύτερα από αυτό εδώ το μοντέλο (και είχε σταματήσει επίσης πρόωρο στην **18η εποχή**).

Αυτό θεωρώ ότι αφείλεται στο ότι το μοντέλο από ένα σημείο και μετά **υπερεκπαιδεύεται** (**overfitting**) στις κατηγορίες που έχει σε μεγάλη ποσότητα, μην μπορώντας να γενικεύσει, με αποτέλεσμα όσο βαθύ και να το κάνουμε να μην αυξάνεται η επίδοση του σχετικά με το πρώτο που είχε πολύ λιγότερα επίπεδα και φίλτρα. Επίσης του πρώτου η εκπαίδευση διήρκησε και **~4 ώρες λιγότερο**.

In [ ]:
cm, y_hat, y_true = confusion_matrix(model3, test_ds)


## **Confusion Matrix του δεύτερου μοντέλου**

Παρακάτω έχουμε το **confusion matrix** και το **f1_score** του μοντέλου μας στο **test set**, το οποίο είναι όσο κακό το περιμέναμε. 

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_true, y_hat, average='micro')

In [ ]:
heat_confmatrix(cm.numpy())

# **Εκπαίδευση προεκπαιδευμένου μοντέλου, στο dataset μας.**
Έτρεξε για **1 ώρα και 55 λεπτά**, τις 5 εποχές, και μας έδωσε accuracy πολύ καλύτερο απ τα προηγούμενα μοντέλα που έτρεξαν για 20 εποχές

**loss: 0.2828 accuracy: 0.9440**

Οπότε παρατηρούμε πως όντως ένα **πιο βαθύ δίκτυο** το οποίο είχε **5,330,571** παραμέτρους σε αντίθεση με τους **7,671,337** που είχε το προηγούμενο μοντέλο μας, εκπαιδεύεται πολύ καλύτερα λόγω βάθους (**μεγάλου αριθμού επιπέδων**) αλλά **κυρίως** λόγω της **εμπειρίας** του πάνω σε **image datasets**. Αυτό γιατί το **EfficientNetB0** έχει προ-εκπαιδευτεί σε ένα μεγάλο σύνολο δεδομένων και έχει ρυθμιστεί λεπτομερώς στη συγκεκριμένη εργασία που προσπαθούμε να επιλύσουμε. Αυτό σημαίνει ότι έχει ήδη ένα ισχυρό θεμέλιο γνώσης και μπορεί να είναι σε θέση να μάθει γρηγορότερα και να αποδώσει καλύτερα από ένα μοντέλο που έχει αρχικοποιηθεί τυχαία.

In [ ]:
model4 = keras.applications.efficientnet.EfficientNetB0()

model4.summary()

In [ ]:
model4.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model4.fit(train_ds, validation_data=val_ds, batch_size=32, epochs=5)
score = model4.evaluate(val_ds)

In [ ]:
cm, y_hat = confusion_matrix(model4, test_ds)

## **Confusion Matrix των προβλέψεων του EfficientNetB0**
Πέρα από 29 δείγματα που τα πρόβλεψε ως **NORM**, ενώ ήταν **ADI**, και τα 22 δείγματα που τα πρόβλεψε ως **MUS** ενώ ήταν **STR**, σε όλα τα άλλα, εκτός ελάχιστων αστοχιών, τα έχει πάει καταπληκτικά.

Το accuracy του στο test set είναι (1318 / 1408): **0.936**



**Σημ**: Για να μην το ξανατρέχει επειδή το ολοκλήρωσα άλλη μέρα, έκανα την πράξη της διαίρεσης της διαγωνίου με το σύνολο, με το χέρι.

In [ ]:
heat_confmatrix(cm.numpy())